In [46]:
import tensorflow as tf
import numpy as np
import pandas as pd
import copy
from matplotlib import pyplot as plt
import seaborn as sns

import world_model
import plan_optimizer as po
import planning_cases

sns.set(style="whitegrid")
sns.set_context(context="paper")

In [47]:
df = pd.read_parquet("data/angle_test_0.parquet", engine="pyarrow")
print(df.describe())
print(df.info())
print(df.head())
print(df["i"].unique())
df.tail()

        init_angle  init_speed         step        theta    theta_dot       i
count  1989.000000      1989.0  1989.000000  1989.000000  1989.000000  1989.0
mean      0.000000         0.0    25.000000     0.022505    -0.086345     0.0
std      18.712992         0.0    14.723303     1.638066     1.548157     0.0
min     -30.000000         0.0     0.000000    -4.294714    -5.219311     0.0
25%     -15.000000         0.0    12.000000    -0.188320    -0.142680     0.0
50%       0.000000         0.0    25.000000    -0.107890    -0.043754     0.0
75%      15.000000         0.0    38.000000     0.114213     0.023096     0.0
max      30.000000         0.0    50.000000     4.698498     5.376655     0.0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1989 entries, 0 to 1988
Data columns (total 6 columns):
init_angle    1989 non-null int64
init_speed    1989 non-null int64
step          1989 non-null int64
theta         1989 non-null float64
theta_dot     1989 non-null float64
i             1989

,init_angle,init_speed,step,theta,theta_dot,i
1984,30,0,46,2.610459,-2.032233,0
1985,30,0,47,2.512842,-1.952350,0
1986,30,0,48,2.452279,-1.211249,0
1987,30,0,49,2.400567,-1.034243,0
1988,30,0,50,2.359169,-0.827959,0


In [48]:
condition_observations = len(df["init_angle"].unique()) * len(df["step"].unique())
print(condition_observations)
conditions = [x * condition_observations for x in [["None"], ["First"], ["Last"]]]
conditions = [condition for sublist in conditions for condition in sublist]
len(conditions)


663


1989

In [49]:
df["condition"] = conditions

In [50]:
print(df.head())
print(df.tail())


   init_angle  init_speed  step     theta  theta_dot  i condition
0         -30           0     0 -0.523599   0.000000  0      None
1         -30           0     1 -0.527349  -0.075000  0      None
2         -30           0     2 -0.534970  -0.152433  0      None
3         -30           0     3 -0.546710  -0.234795  0      None
4         -30           0     4 -0.562945  -0.324705  0      None
      init_angle  init_speed  step     theta  theta_dot  i condition
1984          30           0    46  2.610459  -2.032233  0      Last
1985          30           0    47  2.512842  -1.952350  0      Last
1986          30           0    48  2.452279  -1.211249  0      Last
1987          30           0    49  2.400567  -1.034243  0      Last
1988          30           0    50  2.359169  -0.827959  0      Last


In [51]:
df_list = [df]
for i in range(1, 8):
    new_df = pd.read_parquet(f"data/angle_test_{i}.parquet", engine="pyarrow")
    new_df["condition"] = conditions
    df_list.append(new_df)
df = pd.concat(df_list, ignore_index=True)
print(df.describe())
print(df.info())
print(df.isna().count())
print(df["i"].unique())
print(df.head())
df.tail()

         init_angle  init_speed          step         theta     theta_dot  \
count  15912.000000     15912.0  15912.000000  15912.000000  15912.000000   
mean       0.000000         0.0     25.000000      0.022505     -0.086345   
std       18.708875         0.0     14.720064      1.637706      1.547816   
min      -30.000000         0.0      0.000000     -4.294714     -5.219311   
25%      -15.000000         0.0     12.000000     -0.188320     -0.142680   
50%        0.000000         0.0     25.000000     -0.107890     -0.043754   
75%       15.000000         0.0     38.000000      0.114213      0.023096   
max       30.000000         0.0     50.000000      4.698498      5.376655   

                 i  
count  15912.00000  
mean       3.50000  
std        2.29136  
min        0.00000  
25%        1.75000  
50%        3.50000  
75%        5.25000  
max        7.00000  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15912 entries, 0 to 15911
Data columns (total 7 columns):
init_angl

,init_angle,init_speed,step,theta,theta_dot,i,condition
15907,30,0,46,2.610459,-2.032233,7,Last
15908,30,0,47,2.512842,-1.952350,7,Last
15909,30,0,48,2.452279,-1.211249,7,Last
15910,30,0,49,2.400567,-1.034243,7,Last
15911,30,0,50,2.359169,-0.827959,7,Last


In [52]:
df.describe()

,init_angle,init_speed,step,theta,theta_dot,i
count,15912.000000,15912.0,15912.000000,15912.000000,15912.000000,15912.00000
mean,0.000000,0.0,25.000000,0.022505,-0.086345,3.50000
std,18.708875,0.0,14.720064,1.637706,1.547816,2.29136
min,-30.000000,0.0,0.000000,-4.294714,-5.219311,0.00000
25%,-15.000000,0.0,12.000000,-0.188320,-0.142680,1.75000
50%,0.000000,0.0,25.000000,-0.107890,-0.043754,3.50000
75%,15.000000,0.0,38.000000,0.114213,0.023096,5.25000
max,30.000000,0.0,50.000000,4.698498,5.376655,7.00000


In [53]:
df_list = [df]
for i in range(0, 7):
    new_df = pd.read_parquet(f"data/angle_test_2_{i}.parquet", engine="pyarrow")
    new_df["i"] = new_df["i"] + len(df["i"].unique())
    new_df["condition"] = conditions
    print(new_df.head())
    df_list.append(new_df)
df = pd.concat(df_list, ignore_index=True)
print(df.describe())
print(df.info())
print(df.isna().sum())
print(df["i"].unique())
print(df.head())
df.tail()

   init_angle  init_speed  step     theta  theta_dot  i condition
0         -30           0     0 -0.523599   0.000000  8      None
1         -30           0     1 -0.527349  -0.075000  8      None
2         -30           0     2 -0.534970  -0.152433  8      None
3         -30           0     3 -0.546710  -0.234795  8      None
4         -30           0     4 -0.562945  -0.324705  8      None
   init_angle  init_speed  step     theta  theta_dot  i condition
0         -30           0     0 -0.523599   0.000000  9      None
1         -30           0     1 -0.527349  -0.075000  9      None
2         -30           0     2 -0.534970  -0.152433  9      None
3         -30           0     3 -0.546710  -0.234795  9      None
4         -30           0     4 -0.562945  -0.324705  9      None
   init_angle  init_speed  step     theta  theta_dot   i condition
0         -30           0     0 -0.523599   0.000000  10      None
1         -30           0     1 -0.527349  -0.075000  10      None
2      

,init_angle,init_speed,step,theta,theta_dot,i,condition
29830,30,0,46,2.610459,-2.032233,14,Last
29831,30,0,47,2.512842,-1.952350,14,Last
29832,30,0,48,2.452279,-1.211249,14,Last
29833,30,0,49,2.400567,-1.034243,14,Last
29834,30,0,50,2.359169,-0.827959,14,Last


In [54]:
df_list = [df]
for i in range(0, 7):
    new_df = pd.read_parquet(f"data/angle_test_3_{i}.parquet", engine="pyarrow")
    new_df["i"] = new_df["i"] + len(df["i"].unique())
    new_df["condition"] = conditions
    print(new_df.head())
    df_list.append(new_df)
df = pd.concat(df_list, ignore_index=True)
print(df.describe())
print(df.info())
print(df.isna().sum())
print(df["i"].unique())
print(df.head())
df.tail()

   init_angle  init_speed  step     theta  theta_dot   i condition
0         -30           0     0 -0.523599   0.000000  15      None
1         -30           0     1 -0.527349  -0.075000  15      None
2         -30           0     2 -0.534970  -0.152433  15      None
3         -30           0     3 -0.546710  -0.234795  15      None
4         -30           0     4 -0.562945  -0.324705  15      None
   init_angle  init_speed  step     theta  theta_dot   i condition
0         -30           0     0 -0.523599   0.000000  16      None
1         -30           0     1 -0.527349  -0.075000  16      None
2         -30           0     2 -0.534970  -0.152433  16      None
3         -30           0     3 -0.546710  -0.234795  16      None
4         -30           0     4 -0.562945  -0.324705  16      None
   init_angle  init_speed  step     theta  theta_dot   i condition
0         -30           0     0 -0.523599   0.000000  17      None
1         -30           0     1 -0.527349  -0.075000  17      

,init_angle,init_speed,step,theta,theta_dot,i,condition
43753,30,0,46,2.610459,-2.032233,21,Last
43754,30,0,47,2.512842,-1.952350,21,Last
43755,30,0,48,2.452279,-1.211249,21,Last
43756,30,0,49,2.400567,-1.034243,21,Last
43757,30,0,50,2.359169,-0.827959,21,Last


In [55]:
print(len(df["i"].unique()))

22


In [56]:
df.to_parquet("data/angle_test_result_22.parquet", engine="pyarrow")